#OAuth Exercise

In this exercise we will try to scrape twitter data and do a tf-idf analysis on that (src-uwes twitter analysis). We will need OAuth authentication, and we will follow a similar approach as detailed in the yelp analysis notebook. 

In [1]:
import oauth2 as oauth
import urllib2 as urllib
import json, operator
import numpy as np
import pandas as pd

We will now need twitter api access. The following steps as available online will help you set up your twitter account and access the live 1% stream.

1. Create a twitter account if you do not already have one.
2. Go to https://dev.twitter.com/apps and log in with your twitter credentials.
3. Click "Create New App"
4. Fill out the form and agree to the terms. Put in a dummy website if you don't have one you want to use.
5. On the next page, click the "API Keys" tab along the top, then scroll all the way down until you see the section "Your Access Token"
6. Click the button "Create My Access Token". You can Read more about Oauth authorization online. 

Save the details of api_key, api_secret, access_token_key, access_token_secret in your vaule directory and load it in the notebook as shown in yelpSample notebook.

In [2]:
# We need to define the following variables
#api_key = #<get api key> 
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>

#defining them right here is not safe. insteadm create a file in a different directory
# (I use ~/VaultDSE) and in it put a file called, say, twitterkeys.py whose
# content is:
#api_key = #<get api key>
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>
#
#def getkeys():
#    return api_key,api_secret,access_token_key,access_token_secret

# then use the following commands

import sys
sys.path.append('/home/sadat/Documents/DSE/VaultDSE')
import twitterKeys
api_key,api_secret,access_token_key,access_token_secret=twitterKeys.getkeys()

_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"

http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

Below is a twitter request method which will use the above user logins to sign, and open a twitter stream request

In [3]:
def getTwitterStream(url, method, parameters):
  req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

  req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

  headers = req.to_header()

  if http_method == "POST":
    encoded_post_data = req.to_postdata()
  else:
    encoded_post_data = None
    url = req.to_url()

  opener = urllib.OpenerDirector()
  opener.add_handler(http_handler)
  opener.add_handler(https_handler)

  response = opener.open(url, encoded_post_data)

  return response





We can use the above function to request a response as follows

In [4]:
#Now we will test the above function for a sample data provided by twitter stream here -  
url = "https://stream.twitter.com/1/statuses/sample.json"

In [5]:
parameters = []
response = getTwitterStream(url, "GET", parameters)

Write a function which will take a url and return the top 10 lines returned by the twitter stream

** Note ** The response returned needs to be intelligently parsed to get the text data which correspond to actual tweets. This part can be done in a number of ways and you are encouraged to try different approaches to parse the response data.

In [6]:
def fetchData(url):
    response = getTwitterStream(url, "GET", [])
    lines = response.read()
    j = json.loads(lines)
    h = j['statuses']
    print 'Stream: ',url.split('/')[-1][14:], '\n'
    for i in range(10):
        try:
            print i+1
            print h[i]['text'],'\n'
        except:
            continue
    print '\n\n\n'

In [7]:
list_query = ['UCSD', 'Donald Trump', 'Syria']

for search_query in list_query:
    #We can also request twitter stream data for specific search parameters as follows
    url= "https://api.twitter.com/1.1/search/tweets.json?q="+search_query
    fetchData(url)

Stream:  UCSD 

1
RT @Keysight: Keysight collaborates with #UCSD to demonstrate world’s first #5G, 100-200 meter communication link up to 2 Gbps: https://t.c… 

2
World's First 5G, 100 To 200 Meter Comms Link Up To 2 Gbps Demo'd By Keysight Technologies + UCSD https://t.co/n3aow4n1Iq #science 

3
RT @augmentl: HTC Vive Tour Hits NC State and UCSD Universities this Week https://t.co/ppEZMnwE1H #virtualreality #vr 

4
RT @FamilyoftheYear: San Diego 🤑 see ya tonight @ the loft #ucsd 

5
Climate models underestimate the observed deoxygenation of oxygen minimum zones - Corinne Le Quere #OceanforClimate @Scripps_Ocean #ucsd 

6
RT @Keysight: Keysight collaborates with #UCSD to demonstrate world’s first #5G, 100-200 meter communication link up to 2 Gbps: https://t.c… 

7
With organizers and speakers of the Tara Expeditions ocean science event @Scripps_Ocean #OceanforClimate #ucsd https://t.co/fUC1aslJT1 

8
RT @ArchivosEst: They still draw pictures. Drawings made by Spanish children during th

Call the fetchData function to fetch latest live stream data for following search queries and output the first 5 lines

1. "UCSD"
2. "Donald Trump"
3. "Syria"

### TF-IDF###

tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is among the most regularly used statistical tool for word cloud analysis. You can read more about it online (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

We base our analysis on the following

1. The weight of a term that occurs in a document is simply proportional to the term frequency
2. The specificity of a term can be quantified as an inverse function of the number of documents in which it occurs

For this question we will perform tf-idf analysis o the stream data we retrieve for a given search parameter. Perform the steps below

1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt
2. load the saved file and output the count of occurrences for each term. This will be your term frequency
3. Calculate the inverse document frequency for each of the term in the output above.
4. Multiply the term frequency for each of the term by corresponding inverse document frequency.
5. Sort the terms in the descending order based on their term freq/inverse document freq scores 
6. Print the top 10 terms.

In [8]:
#1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt
writer = open('twitterStream.txt', 'a') 
url= "https://api.twitter.com/1.1/search/tweets.json?q="+"syria"
response = getTwitterStream(url, "GET", [])
lines = response.read()
j = json.loads(lines)
h = j['statuses']
for i in range(200):
    try:
        writer.write(h[i]['text']+'\n')
    except:
        continue
writer.close()

print 'Twitter Stream file generated'

Twitter Stream file generated


In [16]:
#2. load the saved file and output the count of occurrences for each term. This will be your term frequency

def tf(name):
    '''Term Frequency'''
    char = '.,?"'
    text = open(name, 'r')
    line = text.read()
    text.close()
    word_list=line.lower().split()
    count_dict = {}
    for word in word_list:
        if word[-1] in char:
            word = word[:-1]
        if word not in count_dict:
            count_dict[word]=0
    for word in word_list:
        if word[-1] in char:
            word = word[:-1]
        count_dict[word]+=1
    return count_dict

name = 'twitterStream.txt'
tf = tf(name)

print 'Term Frequency:\n\n'
print tf

Term Frequency:


{'and': 1, 'cons': 1, 'https://t.co/mmmkeo3bkc': 1, 'german': 1, 'pros': 1, 'over': 1, 'https://t.co/cqewbkjnot': 1, 'mission': 1, 'wednesday': 1, 'vote': 1, 'us': 1, 'in': 2, 'go': 1, 'says': 1, 'breaking:': 1, "'islamic": 1, 'agrees': 3, 'https://t.co/2xssucuxml': 1, 'rt': 2, 'guess': 1, 'https://t.co/ura1lpqavo': 1, 'sadly': 1, 'i': 1, 'no': 1, "it's": 1, 'support': 1, '-': 1, 'should': 1, 'better': 1, 'to': 2, 'out': 1, 'war': 1, 'seeks': 1, 'pm': 1, 'ken': 3, '@conflicts:': 1, "state'": 1, 'on': 1, '@dwnews': 1, 'approves': 1, 'who': 3, 'but': 1, 'video:': 1, 'hear': 1, 'opinion': 1, 'not': 1, 'cabinet': 2, 'with': 3, 'is': 1, ':s': 3, 'well': 1, '#syria': 1, 'being': 1, 'we': 1, 'https://t.co/yadykuhdbz': 1, '@skynews:': 1, 'against': 1, 'public': 1, 'this': 1, 'bbc': 1, 'https://t.co/zv8kovbuwo': 1, 'time': 1, 'syria': 3, 'the': 1, 'asked': 1}


In [23]:
#3. Calculate the inverse document frequency for each of the term in the output above.

def idf(name):
    '''Inverse Document Frequency'''
    docs = open(name, 'r')
    tot_docs = len(docs.readlines())
    count_dict = {}
    unique = []
    docs.close()
    
    #Get all unique terms
    docs = open(name, 'r')
    char = '.,?"'
    text_list = docs.read().lower().split()
    for word in text_list:
        if word[-1] in char:
            word = word[:-1]
        if word not in unique:
            unique.append(word)
            count_dict[word] = 0
    docs.close()
    
    #Term count in each doc
    docs = open(name, 'r')
    for line in docs.readlines():
        new_line = []
        for word in line.lower().split():
            if word[-1] in char:
                word = word[:-1]
            new_line.append(word)
        for term in unique:
            if term[-1] in char:
                term = term[:-1]
            if term in new_line:
                count_dict[term] += 1
            else:
                pass    
    docs.close()
    
    #IDF calculation
    for key in count_dict:
        count_dict[key] = np.log10(float(tot_docs) / float(count_dict[key]))
    
    return count_dict
        
        
name = 'twitterStream.txt'    
idf = idf(name)
print 'Inverse Document Frequency:\n\n'
print idf

Inverse Document Frequency:


{'and': 0.84509804001425681, 'cons': 0.84509804001425681, 'https://t.co/mmmkeo3bkc': 0.84509804001425681, 'german': 0.84509804001425681, 'pros': 0.84509804001425681, 'over': 0.84509804001425681, 'https://t.co/cqewbkjnot': 0.84509804001425681, 'mission': 0.84509804001425681, 'wednesday': 0.84509804001425681, 'vote': 0.84509804001425681, 'us': 0.84509804001425681, 'in': 0.54406804435027567, 'go': 0.84509804001425681, 'says': 0.84509804001425681, 'breaking:': 0.84509804001425681, "'islamic": 0.84509804001425681, 'agrees': 0.36797678529459443, 'https://t.co/2xssucuxml': 0.84509804001425681, 'rt': 0.54406804435027567, 'guess': 0.84509804001425681, 'https://t.co/ura1lpqavo': 0.84509804001425681, 'sadly': 0.84509804001425681, 'i': 0.84509804001425681, 'no': 0.84509804001425681, "it's": 0.84509804001425681, 'support': 0.84509804001425681, '-': 0.84509804001425681, 'should': 0.84509804001425681, 'better': 0.84509804001425681, 'to': 0.54406804435027567, 'out': 0.845

In [18]:
#4. Multiply the term frequency for each of the term by corresponding inverse document frequency.

def tfidf(tf_dict, idf_dict):
    tfidf_dict = {}
    for term in tf_dict.keys():
        tfidf_dict[term] = tf_dict[term] * idf_dict[term]
    return tfidf_dict

tfidf = tfidf(tf, idf)
print 'Term Frequency - Inverse Document Frequency:\n\n'
tfidf

Term Frequency - Inverse Document Frequency:




{'#syria': 0.84509804001425681,
 "'islamic": 0.84509804001425681,
 '-': 0.84509804001425681,
 ':s': 1.1039303558837834,
 '@conflicts:': 0.84509804001425681,
 '@dwnews': 0.84509804001425681,
 '@skynews:': 0.84509804001425681,
 'against': 0.84509804001425681,
 'agrees': 1.1039303558837834,
 'and': 0.84509804001425681,
 'approves': 0.84509804001425681,
 'asked': 0.84509804001425681,
 'bbc': 0.84509804001425681,
 'being': 0.84509804001425681,
 'better': 0.84509804001425681,
 'breaking:': 0.84509804001425681,
 'but': 0.84509804001425681,
 'cabinet': 1.0881360887005513,
 'cons': 0.84509804001425681,
 'german': 0.84509804001425681,
 'go': 0.84509804001425681,
 'guess': 0.84509804001425681,
 'hear': 0.84509804001425681,
 'https://t.co/2xssucuxml': 0.84509804001425681,
 'https://t.co/cqewbkjnot': 0.84509804001425681,
 'https://t.co/mmmkeo3bkc': 0.84509804001425681,
 'https://t.co/ura1lpqavo': 0.84509804001425681,
 'https://t.co/yadykuhdbz': 0.84509804001425681,
 'https://t.co/zv8kovbuwo': 0.845

In [19]:
#5. Sort the terms in the descending order based on their term freq/inverse document freq scores

df = pd.DataFrame(tfidf.items(),columns=['Term','TF-IDF']).sort(ascending=False,columns=['TF-IDF'])

In [22]:
# Print the top 10 terms
df.head(10)

,Term,TF-IDF
33,ken,1.103930
38,who,1.103930
16,agrees,1.103930
46,with,1.103930
49,:s,1.103930
43,syria,1.103930
18,rt,1.088136
45,cabinet,1.088136
28,to,1.088136
11,in,1.088136
